In [1]:
import numpy as np
import csv
import pandas as pd
import ripleyk
import time
import matplotlib.pyplot as plt
import os
import glob

In [2]:
# Choose the radii for the Ripley function
radii = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1] #, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2]


### Complete Ripley function

In [4]:
# THERE IS STILL A HARD CODED LIST OF COLUMN NAMES FOR THE TEN RADII!!

# Define a ripley function returning a dataframe with a column for each radius,
# and a row for each nucleus with the corresponding p-value.

# It is a function of the following:

# 'df', pandas dataframe of (x,y,z) coordinates of spots in a nucleus
# 'radii', list (of length 10) of radii
# 'n_data', number of draws from the null ---> Ideally, this is a multiple of 40
# 'bounding_radius', the (estimated) radius of the sphere bounding the sample (we have to choose this wisely!)
# 'boundary', either 'True' or 'False' indicating if a boundary correction is applied (still does not work ...)
# 'normalise', either 'True' or 'False' indication if normalisation takes place (should say 'False')

# The output of this function is a dictionary with items {A,B,C,P} (see example below how to obtain these outputs):

# Output A = column-ordered pandas dataframe of Ripley K's for the null (descending order)
# Output B = values of the null Ripley K's at 2.5% from the largest (i.e., cut-off for significance)
# Output C = array of ten Ripley estimates for the observed nucleus (for the 10 radii)
# Output P = array of ten p-values for the ten Ripley estimates (Neyman-Pearson)

# For now, only works with a list of 10 radii

def ripley_pvalue(df,radii,n_data,bounding_radius,boundary,normalise):
    
    # Extract the number of spots from the nucleus df
    n_spot = df.shape[0]
    
    # Extract the number of radii (i.e. length of radii_list)
    n_radius = len(radii)
    
    # Extract the max and min coordinate values of spots for the nucleus df
    xMin=(df[0].values.min(0));xMax=(df[0].values.max(0));
    yMin=(df[1].values.min(0));yMax=(df[1].values.max(0));
    zMin=(df[2].values.min(0));zMax=(df[2].values.max(0));

    xDelta = xMax-xMin
    yDelta = yMax-yMin
    zDelta = zMax-zMin
    
    # Generate a null distribution of Ripley K's of uniformly distributed samples with n_spot spots
    null = []
    for i in range(n_data):
        # Obtain three np.arrays with x, y, z coordinates respectively
        xx = xDelta*np.random.uniform(0,1,n_spot)+xMin;#x coordinates of Poisson points
        yy = yDelta*np.random.uniform(0,1,n_spot)+yMin;#y coordinates of Poisson points
        zz = zDelta*np.random.uniform(0,1,n_spot)+zMin;#z coordinates of Poisson points
    
        ripley_null = ripleyk.calculate_ripley(radii,bounding_radius,d1=xx,d2=yy,d3=zz,boundary_correct=boundary, CSR_Normalise=normalise)
        null.append(ripley_null)
    
    # Collect results into a Pandas dataframe
    df_null = pd.DataFrame(null, columns=[str(x) for x in radii])
    
    # Sort each column in descending order (hence the [::-1] after np.sort())
    df_null_ordered = pd.DataFrame(np.sort(df_null.values, axis=0)[::-1], index=df_null.index, columns=df_null.columns)
    df_median = df_null_ordered.iloc[np.floor(n_data / 2).astype(int)]

    # For each radius, find the value 2.5% away from the largest
    null_cutoffs = []
    for i in range(n_radius):
        significance_cut = np.floor(n_data / 40).astype(int)
        null_cutoffs.append(df_null_ordered.iloc[significance_cut][i])
    
    null_cuts = np.asarray(null_cutoffs)
    # Apply Ripley K to the data of the nucleus
    ripley_result = ripleyk.calculate_ripley(radii,bounding_radius,d1=df[0],d2=df[1],d3=df[2],boundary_correct=boundary, CSR_Normalise=normalise)
    ripley_result = np.asarray(ripley_result)
    
    # Compute the p-values for each (nucleus,radius) pair
    # The choice of p-value is OPTIMISTIC, so it is biased towards significance
    # If still not significant, this strengthens the argument for true non-significance.
    pvals = []
    for i in range(n_radius):
        if ripley_result[i] == null_cuts[i]:
            idx = df_null_ordered.index[(df_null_ordered.iloc[:,i] == ripley_result[i])]
            n_cut = n_data / 40
            idx_cut = [x for x in idx if x < n_cut]
            
            above_cutoff = len(idx_cut)
            equal_to_cutoff = (df_null_ordered.iloc[:,i] == ripley_result[i]).sum()
            coin_prob = above_cutoff / equal_to_cutoff
            
            coin = np.random.binomial(1,coin_prob,size = None)
            if coin == 1:
                if np.min(idx) < n_data/2:
                    pvalue = (np.min(idx) / n_data) * 2
                else:
                    pvalue = ( (n_data - np.min(idx) ) / n_data) * 2
            else:
                if np.max(idx) < n_data/2:
                    pvalue = (np.max(idx) / n_data) * 2
                else:
                    # If ripley_result[i] == null_cuts[i] *and* we have lost the coin toss, then it's over
                    pvalue = 1
                
            pvals.append(pvalue)
        
        # Are these p-value computations correct?
        elif ripley_result[i] > null_cuts[i]:
            idx = df_null_ordered.index[df_null_ordered.iloc[:,i] >= ripley_result[i]]
            # Test if list is empty
            if idx.empty:
                pvalue = (1 / n_data) * 2
            else:
                pvalue = (np.min(idx) / n_data) * 2
            
            pvals.append(pvalue)
        else:
            idx = df_null_ordered.index[df_null_ordered.iloc[:,i] <= ripley_result[i]]
            if np.min(idx) < n_data/2:
                pvalue = (np.min(idx) / n_data) * 2
            else:
                pvalue = ( (n_data - np.min(idx) ) / n_data ) * 2
            
            pvals.append(pvalue)
    
    pvals = np.asarray(pvals)
    
    return pd.DataFrame(np.transpose(np.array([radii, df_median, null_cuts, ripley_result, pvals])), columns = ['Radii', 'Mean', 'sigcutoff', 'Observed', 'pvals'])


In [80]:
os.chdir('M:\Karin\PhD\Python\KP107.48_G326_G328_RipleyK_ 2021_10_04\Input_Ripley_K')
extension = 'csv'
files = [i for i in glob.glob('*.{}'.format(extension))]

In [83]:
results = pd.DataFrame()
for i in files:
    input = pd.read_csv(i, sep=",", header=None)
    output = ripley_pvalue(input,radii,50000,5.0,False,False)
    output['filename'] = i
    results = pd.concat([results, output])

In [84]:
#results.to_csv('KP107.48_output_50k_3D.csv', index=False)

In [ ]:
head(results)

In [5]:
os.getcwd()

'/Users/sjoerdbeentjes/Dropbox/ChancellorsFellowship/Projects/ecDNA_Karin'

### Original code

In [118]:
n_data = 5000
results = []

for i in range(n_data):
    # Obtain three np.arrays with x, y, z coordinates respectively
    xx = xDelta*np.random.uniform(0,1,n_spot)+xMin;#x coordinates of Poisson points
    yy = yDelta*np.random.uniform(0,1,n_spot)+yMin;#y coordinates of Poisson points
    zz = zDelta*np.random.uniform(0,1,n_spot)+zMin;#z coordinates of Poisson points
    
    ripley_estimates = ripley_k(xx,yy,zz)
    results.append(ripley_estimates)

# Collect results into a Pandas dataframe
df_results = pd.DataFrame(results, columns=['0.1um', '0.2um', '0.3um','0.4um', '0.5um', '0.6um','0.7um', '0.8um', '0.9um','1.0um'])

# Sort each column in descending order (hence the [::-1] after np.sort())
df_ordered = pd.DataFrame(np.sort(df_results.values, axis=0)[::-1], index=df_results.index, columns=df_results.columns)

df_ordered.iloc[1:20,:]

#df2 = pd.DataFrame(results)
#df2.to_csv('M:/Karin/PhD/Python/KP107.51_G325_RipleyK_10k/Random_Nucleus_Ripley_K/G325_13c_random.csv',index=False)


,0.1um,0.2um,0.3um,0.4um,0.5um,0.6um,0.7um,0.8um,0.9um,1.0um
1,0.098708,0.296125,0.493542,0.690959,0.987084,1.283209,1.678043,2.270293,2.961252,3.652211
2,0.098708,0.296125,0.394834,0.690959,0.888376,1.283209,1.678043,2.270293,2.862544,3.454794
3,0.098708,0.197417,0.394834,0.592250,0.888376,1.283209,1.678043,2.270293,2.763836,3.356086
4,0.098708,0.197417,0.394834,0.592250,0.888376,1.283209,1.579335,2.270293,2.665127,3.356086
5,0.098708,0.197417,0.394834,0.592250,0.888376,1.283209,1.579335,2.171585,2.665127,3.257378
6,0.098708,0.197417,0.394834,0.592250,0.888376,1.184501,1.579335,2.171585,2.566419,3.257378
7,0.098708,0.197417,0.394834,0.592250,0.789667,1.184501,1.579335,2.171585,2.566419,3.257378
8,0.098708,0.197417,0.394834,0.592250,0.789667,1.184501,1.579335,2.072877,2.566419,3.257378
9,0.098708,0.197417,0.394834,0.592250,0.789667,1.184501,1.480626,2.072877,2.566419,3.158669
10,0.098708,0.197417,0.394834,0.592250,0.789667,1.085793,1.480626,2.072877,2.566419,3.158669
